In [19]:
import numpy as np
from mlp import DenseLayer, MLP

np.random.seed(0)

def mse(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    return np.mean(np.square(y_true - y_pred))

In [20]:
# Input data
rows = 5
cols = 10
input_vector = np.random.normal(size=(rows, cols))
dummy_pred_vector = np.random.normal(size=(rows, cols))

In [21]:
# Instantiate Layer
lyr = DenseLayer(input_dims=input_vector.shape[-1], num_units=2)

In [22]:
# Layer call
lyr_output = lyr(input_vector)
print('input:', input_vector.shape)
print('output:', lyr_output.shape)
print(lyr_output)

input: (5, 10)
output: (5, 2)
[[ 0.92944305 -1.21846771]
 [-0.08838657 -0.81351259]
 [-0.04518326 -0.49300706]
 [ 0.03038661  0.72907292]
 [-1.29596891  0.17739624]]


In [13]:
model = MLP(input_dims=input_vector.shape[0], hidden_units=2, targets=1, loss_function=mse, learning_rate=1e-3)

In [14]:
# Result of forward pass
fwd = model._forward_pass(inputs=input_vector)
print(fwd)
print(fwd.shape)

TypeError: _forward_pass() missing 1 required positional argument: 'loss'

In [ ]:
batch_size = 32
samples = 100
y_batch_indices = np.random.choice(
            a=samples, size=(samples//batch_size, batch_size), replace=False) 

3
[[22 51 90 17 94 76 89 87  8 53 98 55 50 31 26 28 88 79 34 74 39 85 12 16
  83 64 20 67 58 77 25  7]
 [86 36 21 72 84 57  9 11 96 82 68 49  5 14 15 37 19 33 69 78 59 56  4 91
  75  6 95 44 65 62 29 40]
 [71 24 81 41 73 45 13 23 42 52 92 38 63 60  2 46 70 93 32  3  0 80 97 99
   1 30 48 27 47 43 18 10]]
